In [1]:
import pandas as pd
import os
from typing import List
import datetime

In [2]:
# Specify the directory path as a string
directory_path: str = 'data/231102'

# List to store the names of CSV files (each file name is a string)
csv_files: List[str] = [file for file in os.listdir(directory_path) if file.endswith('.csv')]

# List to hold DataFrame objects
all_data: List[pd.DataFrame] = []

# Iterate over the list of file names
for file_name in csv_files:
    file_path: str = os.path.join(directory_path, file_name)  # Construct full file path
    
    df: pd.DataFrame = pd.read_csv(file_path)  # Read CSV into DataFrame
    all_data.append(df)

# Concatenate all DataFrames into a single DataFrame
combined_data: pd.DataFrame = pd.concat(all_data)

In [3]:
combined_data['time'] = pd.to_datetime(combined_data['time']) 

combined_data.head()

,time,device,mac_id,rss,sector_id,zone_id
0,2023-11-02 13:27:30,APPLE,4F-01-2A-82-06-FC,-75,13,13
0,2023-11-02 14:22:55,APPLE,61-1C-C7-A5-82-0E,-67,1,5
1,2023-11-02 14:23:00,APPLE,61-1C-C7-A5-82-0E,-65,1,5
2,2023-11-02 14:23:10,APPLE,61-1C-C7-A5-82-0E,-69,1,5
3,2023-11-02 14:23:20,APPLE,61-1C-C7-A5-82-0E,-79,10,10


In [4]:
specific_date = '2023-11-02'
start_time = '14:00:00'
end_time = '15:00:00'

filtered_data = combined_data[
    (combined_data['time'].dt.date == pd.to_datetime(specific_date).date()) &
    (combined_data['time'].dt.time >= pd.to_datetime(start_time).time()) &
    (combined_data['time'].dt.time <= pd.to_datetime(end_time).time())
]

filtered_data

,time,device,mac_id,rss,sector_id,zone_id
0,2023-11-02 14:22:55,APPLE,61-1C-C7-A5-82-0E,-67,1,5
1,2023-11-02 14:23:00,APPLE,61-1C-C7-A5-82-0E,-65,1,5
2,2023-11-02 14:23:10,APPLE,61-1C-C7-A5-82-0E,-69,1,5
3,2023-11-02 14:23:20,APPLE,61-1C-C7-A5-82-0E,-79,10,10
4,2023-11-02 14:23:25,APPLE,61-1C-C7-A5-82-0E,-73,1,5
...,...,...,...,...,...,...
3,2023-11-02 14:58:25,APPLE,7F-24-98-B5-13-30,-83,11,15
4,2023-11-02 14:58:50,APPLE,7F-24-98-B5-13-30,-84,25,23
5,2023-11-02 14:59:05,APPLE,7F-24-98-B5-13-30,-77,12,14
6,2023-11-02 14:59:15,APPLE,7F-24-98-B5-13-30,-84,8,8


In [5]:
mac_counts = filtered_data.groupby('mac_id').size()

# Convert the Series to a dictionary (optional)
mac_counts_dict = mac_counts.to_dict()
mac_counts_dict.__len__()

2362

In [6]:
filtered_mac_counts_dict = {key: value for key, value in mac_counts_dict.items()}
# filtered_mac_counts_dict = {key: value for key, value in mac_counts_dict.items() if value >= 120}
filtered_mac_counts_dict.__len__()

809

In [7]:
mac_addresses_to_keep = list(filtered_mac_counts_dict.keys())

filtered_data = filtered_data[filtered_data['mac_id'].isin(mac_addresses_to_keep)]

In [8]:
grouped = filtered_data.groupby('mac_id')['zone_id'].apply(set)

# Convert to dictionary
mac_id_to_zone_id_dict = grouped.to_dict()

mac_id_to_zone_id_dict.__len__()

809

In [9]:
filtered_mac_id_to_zone_id_dict = {
    mac_id: zone_ids
    for mac_id, zone_ids in mac_id_to_zone_id_dict.items()
    # if len(zone_ids) >= 15
}
filtered_mac_id_to_zone_id_dict.__len__()

86

In [10]:
filtered_mac_id_to_zone_id_dict.keys()

dict_keys(['41-0C-98-CD-F8-FC', '41-8E-7D-88-22-3C', '43-95-1B-5D-1A-F6', '43-D7-3D-06-89-BB', '44-0D-52-39-BF-27', '45-81-3D-AD-F7-64', '46-0F-B9-29-05-2E', '46-8E-02-15-BD-5C', '47-48-01-C3-46-6C', '48-5B-3B-DA-16-57', '48-98-DF-96-7D-15', '48-E6-32-E9-C2-DE', '49-CA-50-BD-4A-B8', '49-DC-A5-E3-E9-BC', '4A-52-EC-3F-B0-20', '4B-84-B5-E9-5E-5A', '4C-45-2E-A7-4F-79', '4C-45-C5-56-89-03', '4E-2C-C1-A6-0B-E6', '50-04-32-EE-AA-F5', '50-5D-8A-F0-E1-68', '52-3E-6F-3A-D1-65', '52-F7-03-08-08-E4', '53-52-C6-98-F4-69', '53-A4-78-A9-D8-A2', '53-D1-CB-64-69-25', '54-AB-62-46-57-57', '54-E4-FB-CB-7C-B1', '56-2B-64-88-4F-4D', '56-65-59-4C-09-28', '56-8B-AC-D4-87-A7', '57-68-20-E5-77-FA', '57-CE-A5-4D-61-F5', '58-CA-43-EE-3C-79', '58-F1-D6-73-6F-E8', '59-62-F4-78-14-CA', '59-C1-5F-8E-AE-9A', '5C-1C-AF-22-36-49', '5D-E3-FE-FC-A2-BA', '5D-E8-F9-47-AF-7B', '5E-18-F9-17-3A-DF', '5F-6B-DD-5A-9E-D6', '5F-FA-B7-6F-67-ED', '60-7E-07-46-8C-FE', '60-C0-E5-67-A5-BB', '61-3F-ED-4F-8B-50', '62-F5-91-5C-E4-49', '6

In [11]:
new_filtered_data = filtered_data[filtered_data['mac_id'].isin(filtered_mac_id_to_zone_id_dict.keys())].sort_values(by=['mac_id', 'time'])
new_filtered_data

,time,device,mac_id,rss,sector_id,zone_id
24,2023-11-02 14:00:00,APPLE,41-0C-98-CD-F8-FC,-63,12,14
25,2023-11-02 14:00:05,APPLE,41-0C-98-CD-F8-FC,-63,12,14
26,2023-11-02 14:00:10,APPLE,41-0C-98-CD-F8-FC,-63,12,14
27,2023-11-02 14:00:15,APPLE,41-0C-98-CD-F8-FC,-63,12,14
28,2023-11-02 14:00:20,APPLE,41-0C-98-CD-F8-FC,-63,12,14
...,...,...,...,...,...,...
133,2023-11-02 14:34:20,APPLE,7F-CA-8A-9F-CB-C1,-76,18,18
134,2023-11-02 14:34:25,APPLE,7F-CA-8A-9F-CB-C1,-76,18,18
135,2023-11-02 14:34:30,APPLE,7F-CA-8A-9F-CB-C1,-76,18,18
136,2023-11-02 14:34:35,APPLE,7F-CA-8A-9F-CB-C1,-76,18,18


In [12]:
new_df = pd.DataFrame(columns=['mac_id', 'first_zone_id', 'last_zone_id', 'first_time', 'last_time'])

# Precompute groupby results
grouped = new_filtered_data.groupby('mac_id')
firsts = grouped.first()
lasts = grouped.last()

# Build list of dictionaries
data_list = []
for mac_id in filtered_mac_id_to_zone_id_dict.keys():
    if mac_id in firsts.index and mac_id in lasts.index:
        data_list.append({
            'mac_id': mac_id,
            'first_zone_id': firsts.loc[mac_id]['zone_id'],
            'last_zone_id': lasts.loc[mac_id]['zone_id'],
            'first_time': firsts.loc[mac_id]['time'],
            'last_time': lasts.loc[mac_id]['time']
        })

# Create DataFrame from list
new_df = pd.DataFrame(data_list)

# Display the new DataFrame
new_df


,mac_id,first_zone_id,last_zone_id,first_time,last_time
0,41-0C-98-CD-F8-FC,14,8,2023-11-02 14:00:00,2023-11-02 14:16:20
1,41-8E-7D-88-22-3C,4,19,2023-11-02 14:16:05,2023-11-02 14:52:50
2,43-95-1B-5D-1A-F6,13,1,2023-11-02 14:13:00,2023-11-02 14:42:30
3,43-D7-3D-06-89-BB,18,15,2023-11-02 14:38:55,2023-11-02 14:59:15
4,44-0D-52-39-BF-27,11,6,2023-11-02 14:05:20,2023-11-02 14:34:00
...,...,...,...,...,...
81,7D-5E-27-31-14-E2,13,5,2023-11-02 14:27:25,2023-11-02 14:48:50
82,7E-0D-98-EE-6E-8E,1,23,2023-11-02 14:20:25,2023-11-02 14:40:10
83,7E-76-4C-40-D4-06,23,7,2023-11-02 14:37:20,2023-11-02 15:00:00
84,7F-7A-57-0D-44-E5,23,20,2023-11-02 14:42:35,2023-11-02 15:00:00


In [14]:
for i in new_df.index:
    for ii in new_df.index:
        if new_df.loc[ii]['first_time'] - new_df.loc[i]['last_time'] <= datetime.timedelta(seconds=30) and new_df.loc[ii]['first_time'] - new_df.loc[i]['last_time'] >= datetime.timedelta(minutes=0) :
            if new_df.loc[i]['last_zone_id'] == new_df.loc[ii]['first_zone_id']:
                print(new_df.loc[i]['last_time'], '->', new_df.loc[ii]['first_time'])
                print(new_df.loc[i]['last_zone_id'], '->', new_df.loc[ii]['first_zone_id'])
                print(new_df.loc[i]['mac_id'], '->', new_df.loc[ii]['mac_id'])
                print('------------------------')
        
    

2023-11-02 14:42:35 -> 2023-11-02 14:42:35
7 -> 7
49-CA-50-BD-4A-B8 -> 49-DC-A5-E3-E9-BC
------------------------
2023-11-02 14:42:35 -> 2023-11-02 14:42:35
7 -> 7
49-CA-50-BD-4A-B8 -> 7B-F1-41-5D-CD-0D
------------------------
2023-11-02 14:27:00 -> 2023-11-02 14:27:10
9 -> 9
5F-6B-DD-5A-9E-D6 -> 56-2B-64-88-4F-4D
------------------------
2023-11-02 14:37:00 -> 2023-11-02 14:37:20
23 -> 23
68-11-5E-7A-41-CE -> 7E-76-4C-40-D4-06
------------------------
2023-11-02 14:15:35 -> 2023-11-02 14:16:00
22 -> 22
79-07-EB-BC-85-03 -> 53-A4-78-A9-D8-A2
------------------------
2023-11-02 14:21:35 -> 2023-11-02 14:21:40
14 -> 14
7A-07-C7-DC-A6-B1 -> 49-CA-50-BD-4A-B8
------------------------


In [17]:
new_df.loc[new_df['mac_id'] == '7A-07-C7-DC-A6-B1']

,mac_id,first_zone_id,last_zone_id,first_time,last_time
73,7A-07-C7-DC-A6-B1,3,14,2023-11-02 14:00:00,2023-11-02 14:21:35


In [16]:
new_df.loc[new_df['mac_id'] == '49-CA-50-BD-4A-B8']

,mac_id,first_zone_id,last_zone_id,first_time,last_time
12,49-CA-50-BD-4A-B8,14,7,2023-11-02 14:21:40,2023-11-02 14:42:35


In [18]:
new_df.loc[new_df['mac_id'] == '7B-F1-41-5D-CD-0D']

,mac_id,first_zone_id,last_zone_id,first_time,last_time
76,7B-F1-41-5D-CD-0D,7,20,2023-11-02 14:42:35,2023-11-02 15:00:00


In [19]:
new_df.loc[new_df['mac_id'] == '49-DC-A5-E3-E9-BC']

,mac_id,first_zone_id,last_zone_id,first_time,last_time
13,49-DC-A5-E3-E9-BC,7,9,2023-11-02 14:42:35,2023-11-02 15:00:00


In [20]:
new_df.loc[new_df['mac_id'] == '79-07-EB-BC-85-03']

,mac_id,first_zone_id,last_zone_id,first_time,last_time
71,79-07-EB-BC-85-03,7,22,2023-11-02 14:00:00,2023-11-02 14:15:35


In [21]:
new_df.loc[new_df['mac_id'] == '53-A4-78-A9-D8-A2']

,mac_id,first_zone_id,last_zone_id,first_time,last_time
24,53-A4-78-A9-D8-A2,22,5,2023-11-02 14:16:00,2023-11-02 14:37:30


In [22]:
new_df.loc[new_df['mac_id'] == '5F-6B-DD-5A-9E-D6']

,mac_id,first_zone_id,last_zone_id,first_time,last_time
41,5F-6B-DD-5A-9E-D6,9,9,2023-11-02 14:04:30,2023-11-02 14:27:00


In [23]:
new_df.loc[new_df['mac_id'] == '56-2B-64-88-4F-4D']

,mac_id,first_zone_id,last_zone_id,first_time,last_time
28,56-2B-64-88-4F-4D,9,13,2023-11-02 14:27:10,2023-11-02 14:48:20
